In [1]:
import torch 
import geometric_kernels.torch 
from geometric_kernels.spaces import Hypersphere 
from gpytorch.kernels import ScaleKernel
from mdgp.kernels import GeometricMaternKernel
from mdgp.utils import sphere_random_uniform

INFO: Using numpy backend


In [2]:
x = sphere_random_uniform(1, 3)
space = Hypersphere(2)
batch_shape = torch.Size([2])
kernel = GeometricMaternKernel(space=space, batch_shape=batch_shape)
kernel.lengthscale = torch.tensor([[[20.0], [1.0]]])
kernel(x).evaluate()

tensor([[[1.0000]],

        [[1.0000]]], grad_fn=<PermuteBackward0>)

In [3]:
from geometric_kernels.frontends.pytorch.gpytorch import GPytorchGeometricKernel
from geometric_kernels.kernels import MaternKarhunenLoeveKernel


kernel_unnorm = MaternKarhunenLoeveKernel(space=space, num_levels=25, normalize=False)
kernel_norm = MaternKarhunenLoeveKernel(space=space, num_levels=25, normalize=True)
params = {
    "lengthscale": torch.tensor([[[10.0]], [[1.0]]]),
    "nu": torch.tensor([[[2.5]], [[2.5]]]),
}
params0 = {
    "lengthscale": torch.tensor([[10.0]]),
    "nu": torch.tensor([[2.5]]),
}
params1 = {
    "lengthscale": torch.tensor([[1.0]]),
    "nu": torch.tensor([[2.5]]),
}

print(kernel_unnorm.K(params, x), kernel_unnorm.K(params0, x), kernel_unnorm.K(params1, x))
print(kernel_norm.K(params, x), kernel_norm.K(params0, x), kernel_norm.K(params1, x))

tensor([[[3.5777e+04]],

        [[8.5570e-03]]]) tensor([[35777.3359]]) tensor([[0.0086]])
tensor([[[1.0000]],

        [[1.0000]]]) tensor([[1.0000]]) tensor([[1.0000]])


In [4]:
import lab as B 
from geometric_kernels.lab_extras import from_numpy


spectral_values = kernel_norm._spectrum(kernel_norm.eigenvalues_laplacian, nu=params['nu'], lengthscale=params['lengthscale'])
normalizer = B.sum(
                spectral_values
                * B.cast(
                    B.dtype(spectral_values),
                    from_numpy(
                        spectral_values,
                        kernel_unnorm.eigenfunctions.num_eigenfunctions_per_level,
                    )[:, None],
                ), # [..., num_levels, 1]
                axis=-2
            )[:, 0]
normalizer

tensor([3.5777e+04, 4.9610e-03])